# Итоговая тетрадка с моделью

Загрузка необходимых библиотек

In [1]:
import json
import pandas as pd
import numpy as np

from phik import resources
from phik.binning import bin_data
from phik.report import plot_correlation_matrix
from phik import report

import catboost as cb
import matplotlib.pyplot as plt
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    train_test_split,
)
from catboost import CatBoostClassifier
from sklearn.utils import shuffle

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_auc_score,
)
from scipy.stats import pointbiserialr
import shap

import warnings

warnings.filterwarnings("ignore")

RANDOM_STATE = 42

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Подготовка к обучению

Объявление функций

In [2]:
# Функция для даунсемплинга данных
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=RANDOM_STATE)]
        + [features_ones]
    )
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=RANDOM_STATE)]
        + [target_ones]
    )

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=RANDOM_STATE
    )

    return features_downsampled, target_downsampled


# Функция для нахождения категориальных признаков
cat = ["ctg", "flg", "channel_name", "src_id"]


def cat_columns(columns, cat):
    cat_columns = []
    for col in columns:
        for c in cat:
            if c in col:
                cat_columns.append(col)
    return cat_columns


# Поиск дубликатов в списках
def dubl_list(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

Ниже предоставлены 3 списка признаков, отобранных каждым участников по отдельности. Все 2700 признаков были поделены условно на 3 части, *Николай Каширский* обрабатывал признаки с 0 до 930, *Глеб Якушев* списки с 930 по 1860 и *Николай И.* списки с 1860 и до конца.  
1. Список **a**: отобран *Николаем И*:
- Изучение данных на предмет корреляции с таргетом, используя методы Спирмена и Кендала. 
- Вывод случайного списка из выборок с таргетом 1 и 0 и их изучение. 
- Выписывание странных столбцов. 
- Выделение категориальных признаков для модели Cat Boost. 
- Построение лучшей модели на всей выборке с использованием Grid Search CV. 
- Запрос у модели важных признак, их изучение и добавление понравившихся столбцов. 
- Выяснение причин успешности столбцов с большим количеством пропусков. 
- Тест фичей на новой модели. Выпуск итоговых фичей.

2. Список **nick** отобран *Николаем Каширским*:
- Признаки были разделены на категориальные и непрерывные числовые
- Лишние категориальные признаки были отброшены после корреляционного анализа (Спирман)
- Непрерывные признаки были проверены на наличие пропусков (удалялись если пропусков больше половины)
- После этого непрерывные признаки были проверены на константность (проверил дисперсию)
- И финальной обработкой стал корреляционный анализ (Пирсон). Я считал, если к. корр. между двумя признаками >=0.75 то один из них можно удалять.
- На датасете из итоговых признаков я провел отбор значимых признаков встроенными в LGBM модель методами.
- Было использовано два подхода: downsample и балансировка классов. 2й метод дал лучший результат.
- Всю работу вы можете посмотреть в [тетрадке](https://github.com/YandexhakatonR1/Hakaton_R1/blob/main/nikolai_kashirskii/feature_selection/01-kashirskii-feature-selection.ipynb)

3. Список **gleb** отобран *Глебом Якушевым*:
- выделяем числовые и категориальных фичи, создаем 2 датасета (числовой и категориальный);
- в числовом датасете сначала отбрасываем столбцы, где много пропусков (больше половины), удаляем фичи с константными значениями, удаляем коррелирующие признаки;
- в категориальном датасете: отбрасываем фичи с большим разбросом значений, удаляем коррелирующие признаки, переводим их в 'category';
- объединяем полученные датасеты, дропаем дубликаты, выдиляем список названий категориальных фичей;
- выделяем целевой признак и фичи, обучаем модель LGBM при class_weight = 'balanced' и передавая список категориальных фичей в categorical_feature;
- находим самые важные признаки нашей модели с помощью feature_importances_ , выбираем топ-50.

In [3]:
a = [
    "channel_name",
    "materials_details_16_1_ctg",
    "basic_info_2_0_min",
    "basic_info_0_0_avg",
    "user_devices_30_1_cnt",
    "basic_info_1_0_max",
    "cities_2_0_ctg",
    "user_active_9_1_flg",
    "user_devices_24_1_cnt",
    "type_av_100_0_1_ctg",
    "info_house_10_0_ctg",
    "communication_availability_51_1_flg",
    "materials_details_15_1_ctg",
    "markers_904_1_cnt",
    "movix_app_visits_17_1_cnt",
    "campaigns_369_6_part",
    "markers_933_1_cnt",
    "user_active_23_0_dt",
    "campaigns_357_1_sum",
    "migrant_0_1_flg",
    "markers_925_1_cnt",
    "campaigns_41_6_part",
    "materials_details_21_1_num",
    "campaigns_315_1_sum",
    "agreement_type_0_0_ctg",
    "save_team_answers_21_1_cnt",
    "campaigns_359_1_sum",
    "campaigns_328_1_sum",
    "campaigns_281_1_part",
    "communication_availability_53_1_flg",
    "cities_1_0_ctg",
    "issues_11_6_sum",
    "markers_905_1_cnt",
    "materials_details_22_1_flg",
    "markers_895_1_cnt",
    "user_active_29_1_flg",
    "markers_807_1_cnt",
    "movix_app_visits_62_1_cnt",
    "social_dem_2_0_flg",
    "markers_858_1_cnt",
    "campaigns_40_3_part",
    "campaigns_364_1d6_part",
    "movix_app_visits_24_1_cnt",
    "campaigns_403_3d6_part",
    "user_active_27_0_dt",
    "spas_symptoms_agr_286_12_sum",
    "user_active_10_1_flg",
    "campaigns_324_1_part",
    "user_active_24_0_dt",
    "materials_details_19_1_dt",
]

nick = [
    "tariff_plans_4_1_num",
    "charges_details_12_1_sum",
    "payments_details_35_6_sum",
    "spas_symptoms_agr_7_6_sum",
    "markers_706_1_cnt",
    "payments_details_28_3_sumpct",
    "payments_details_27_1_sumpct",
    "payments_details_29_6_sumpct",
    "balance_details_0_1_num",
    "payments_details_49_6_avg",
    "payments_details_48_3_sum",
    "markers_346_1_cnt",
    "spas_symptoms_agr_18_6_std",
    "arpu_2_6_avg",
    "markers_349_1_cnt",
    "markers_323_1_cnt",
    "markers_476_1_cnt",
    "payments_details_23_3d6_avg",
    "markers_40_1_cnt",
    "markers_310_1_cnt",
    "markers_60_1_cnt",
    "markers_330_1_cnt",
    "markers_333_1_cnt",
    "payments_details_33_1_sum",
    "markers_334_1_cnt",
    "markers_772_1_cnt",
    "markers_59_1_cnt",
    "markers_242_1_cnt",
    "markers_387_1_cnt",
    "tariff_plans_5_1_num",
]

gleb = [
    "user_lifetime_2_1_num",
    "info_house_5_0_num",
    "area_0_0_num",
    "traffic_details_45_3_avg",
    "traffic_details_15_1d3_avg",
    "traffic_details_9_3d6_part",
    "info_house_6_0_num",
    "traffic_details_24_3d6_avg",
    "traffic_details_42_1_avg",
    "traffic_details_63_3_avg",
    "traffic_details_60_1_avg",
    "traffic_details_27_6_avg",
    "traffic_details_5_1d3_part",
    "communication_availability_40_1_ctg",
    "traffic_details_21_3_avg",
    "traffic_details_12_1_avg",
    "spas_symptoms_agr_116_12_avg",
    "tariff_plans_20_1_ctg",
    "spas_symptoms_int_3_1_cnt",
    "traffic_details_0_1_cnt",
    "spas_symptoms_int_17_1_cnt",
    "spas_symptoms_ott_7_1_cnt",
    "spas_symptoms_agr_70_12_sum",
    "user_devices_11_1_cnt",
    "spas_symptoms_agr_214_12_sum",
    "communication_availability_42_1_ctg",
    "spas_symptoms_int_93_1_cnt",
    "communication_availability_35_1_ctg",
    "communication_availability_17_1_flg",
    "spas_symptoms_tv_4_1_cnt",
    "traffic_details_3_1_cnt",
    "user_active_3_1_flg",
    "communication_availability_7_1_ctg",
    "spas_symptoms_agr_92_12_avg",
    "communication_availability_30_1_flg",
    "materials_details_9_1_flg",
    "issues_7_3_sum",
    "communication_availability_4_1_flg",
    "spas_symptoms_int_20_1_cnt",
    "communication_availability_31_1_flg",
    "issues_13_3_sum",
    "scheme_types_0_1_flg",
    "communication_availability_18_1_flg",
    "user_active_2_0_ctg",
    "spas_symptoms_int_0_1_cnt",
    "user_active_0_1_flg",
    "spas_symptoms_int_72_1_cnt",
    "info_house_9_0_flg",
    "issues_6_1_sum",
    "spas_symptoms_int_105_1_cnt",
]

Объединим 3 списка наших признаков и откроем тренировочный датасет

In [4]:
feature_s = a + nick + gleb + ["target"]

In [5]:
feature_s = dubl_list(feature_s)


In [6]:
dataset = pd.read_parquet(
    "dataset_train.parquet", engine="pyarrow", columns=feature_s
)

Удалим дубликаты

In [7]:
dataset.drop_duplicates(inplace=True)


In [8]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 702086 entries, 0 to 702085
Columns: 131 entries, channel_name to target
dtypes: float64(100), int32(20), int64(7), int8(3), object(1)
memory usage: 639.4+ MB


Выделим категориальные признаки:

In [9]:
cat_list = cat_columns(dataset, cat)
cat_list


['channel_name',
 'materials_details_16_1_ctg',
 'cities_2_0_ctg',
 'user_active_9_1_flg',
 'type_av_100_0_1_ctg',
 'info_house_10_0_ctg',
 'communication_availability_51_1_flg',
 'materials_details_15_1_ctg',
 'migrant_0_1_flg',
 'agreement_type_0_0_ctg',
 'communication_availability_53_1_flg',
 'cities_1_0_ctg',
 'materials_details_22_1_flg',
 'user_active_29_1_flg',
 'social_dem_2_0_flg',
 'user_active_10_1_flg',
 'communication_availability_40_1_ctg',
 'tariff_plans_20_1_ctg',
 'communication_availability_42_1_ctg',
 'communication_availability_35_1_ctg',
 'communication_availability_17_1_flg',
 'user_active_3_1_flg',
 'communication_availability_7_1_ctg',
 'communication_availability_30_1_flg',
 'materials_details_9_1_flg',
 'communication_availability_4_1_flg',
 'communication_availability_31_1_flg',
 'scheme_types_0_1_flg',
 'communication_availability_18_1_flg',
 'user_active_2_0_ctg',
 'user_active_0_1_flg',
 'info_house_9_0_flg']

In [10]:
dataset[cat_list] = dataset[cat_list].astype("category")

## Подбор гиперпараметров для LightGBM

Выделим целевой признак и фичи. выделим тестовый датасет размером 20% от исходной выборки.

In [11]:
features = dataset.drop(["target"], axis=1)
target = dataset["target"]

In [12]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=RANDOM_STATE
)

Проведем даунсемплинг

In [13]:
features_downsampled, target_downsampled = downsample(
    features_train, target_train, 0.01
)

In [14]:
model_lgbm = LGBMClassifier(
    random_state=42,
    categorical_feature="name:cat_list",
    class_weight="balanced",
)
model_lgbm.fit(features_downsampled, target_downsampled)

LGBMClassifier(categorical_feature='name:cat_list', class_weight='balanced',
               random_state=42)

In [15]:
%%time

parameters = parameters = {
     'num_iterations': [500, 700, 1000],
     'learning_rate':[0.01, 0.05, 0.1],
    'num_leaves':[7, 15, 31],
    'max_depth' :[5,10,15,25]
}

grid_search = GridSearchCV(estimator=model_lgbm, param_grid=parameters, 
                           cv=5, scoring='roc_auc', n_jobs=-1)

grid_search.fit(features_downsampled, target_downsampled)

print('Best parameters:', grid_search.best_params_)
print('Best parameters:', grid_search.best_score_)

Best parameters: {'learning_rate': 0.01, 'max_depth': 5, 'num_iterations': 1000, 'num_leaves': 7}
Best parameters: 0.7586201670286432
Wall time: 7min 58s


Проверка на тестовых данных

In [16]:
model_lgbm = LGBMClassifier(
    random_state=42,
    categorical_feature="name:cat_list",
    class_weight="balanced",
    learning_rate=0.01,
    max_depth=5,
    num_iterations=1000,
    num_leaves=7,
)
model_lgbm.fit(features_downsampled, target_downsampled)
predictions = model_lgbm.predict(features_test)
roc_auc = roc_auc_score(target_test, predictions)
print("ROC AUC на тестовых данных:", roc_auc)

0.6893669376506595

ROC AUC на тестовых данных: **0.6893**

## Работа с тестовым датасетом

In [17]:
feature_test = feature_s.remove("target")

In [18]:
data_test = pd.read_parquet(
    "features_oot.parquet", engine="pyarrow", columns=feature_s
)

In [19]:
cat_features = cat_columns(data_test, cat)
data_test[cat_list] = data_test[cat_list].astype("category")

In [20]:
features = data_test


In [21]:
features.head()


,channel_name,materials_details_16_1_ctg,basic_info_2_0_min,basic_info_0_0_avg,user_devices_30_1_cnt,basic_info_1_0_max,cities_2_0_ctg,user_active_9_1_flg,user_devices_24_1_cnt,type_av_100_0_1_ctg,...,issues_13_3_sum,scheme_types_0_1_flg,communication_availability_18_1_flg,user_active_2_0_ctg,spas_symptoms_int_0_1_cnt,user_active_0_1_flg,spas_symptoms_int_72_1_cnt,info_house_9_0_flg,issues_6_1_sum,spas_symptoms_int_105_1_cnt
0,3,NaN,-0.364331,-0.364331,-0.431207,-0.364331,52,0,-0.365287,2,...,NaN,0,1,1,0.349689,0,-0.092503,1,NaN,-0.162509
1,3,NaN,-0.813950,-0.813950,-0.431207,-0.813950,52,0,-0.365287,2,...,NaN,0,1,1,-1.435237,0,-0.092503,0,NaN,-0.162509
2,3,NaN,-0.591468,-0.591468,-0.431207,-0.591468,52,1,-0.365287,2,...,-0.219283,0,1,2,0.349689,0,-0.092503,1,-0.32564,-0.162509
3,3,NaN,-1.048375,-1.048375,-0.431207,-1.048375,52,1,-0.365287,2,...,NaN,0,1,2,0.349689,0,-0.092503,1,NaN,-0.162509
4,3,NaN,-1.059711,-1.059711,-0.431207,-1.059711,52,1,-0.365287,2,...,-0.219283,0,1,2,0.349689,0,-0.092503,0,-0.32564,-0.162509


In [22]:
features.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60661 entries, 0 to 60660
Columns: 130 entries, channel_name to spas_symptoms_int_105_1_cnt
dtypes: category(32), float64(98)
memory usage: 50.0 MB


In [23]:
target_test = model_lgbm.predict_proba(features)


In [24]:
target_test = target_test[:, 1]
target_test = pd.DataFrame(target_test)

In [25]:
target_test.reset_index(inplace=True)

In [26]:
target_test.columns = ["id", "target"]

In [27]:
target_test


,id,target
0,0,0.659200
1,1,0.632967
2,2,0.741170
3,3,0.619325
4,4,0.732824
...,...,...
60656,60656,0.651724
60657,60657,0.568692
60658,60658,0.732197
60659,60659,0.748419


In [28]:
target_test.to_csv("target_test.csv", index=False)